# Modelando a la Pandemia 

En este proyecto vamos a estudiar y analizar los datos mundiales de la pandemia COVID-19

Importamos las librerías 

In [1]:
import warnings
import numpy as np
import pandas as pd
import pylab as plt
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import seaborn as sns

In [2]:
warnings.filterwarnings("ignore")

Los datos con los que vamos a trabajar son los utilizados del hub de Our World in Data 

Link: 
https://github.com/owid/covid-19-data/tree/master/public/data

Este dataset que vamos a utilizar posee de features los cuales vamos a filtrar en su mayor parte. La razón para utilizar esta base de datos, con respecto a otras tales como las del CDC, es por que su gran cantidad de variables e indicadores reportados, los cuales van a servir para, filtrarlos y transformar los features en la mayoría de los casos, y por ende poder seleccionar los indicadores necesarios para mensurar la politica seleccionada para conbatir el Covid-19, tratando de utilizar de valernos lo más posible de ratios utilizados en el sector de salud para poder evitar la mayor cantidad de sesgos posibles en la comparación entre países.

Por poner un ejemplo, la cantidad de tests por habitantes en cada país aun la cantidad de test por habitantes es muy baja y obviamente, si el país tiene menor cantid,ad de chequeos su relación de contagiados por habitante va a ser menor, pero no por ello es que existen menos contagiados


## Análisis Descriptivo

In [3]:
dataset2=pd.read_csv("owid-covid-data.csv")
dataset2.head()

FileNotFoundError: ignored

In [ ]:
dataset2.shape

In [ ]:
dataset2.index

In [ ]:
dataset2.columns

In [ ]:
print(dataset2.info())

In [ ]:
for col in dataset2.columns:
    nulos=dataset2[col].isna().sum()
    print(col, "{:.2f}%".format(nulos*100/135153))

Vamos a observar los paises que se encuentran en el dataset.

Notamos tambien que hay instancias que se denominan, por ejemplo "High income", "Lower middle income", "European Union", "Europe", esto genera que haya muchos missing values en datos sensibles para el anáisis por lo que vamos a identificarlas y eliminarlas, ya hemos identificado que se denominan OWID.

In [ ]:
df2 = pd.DataFrame(dataset2.location.value_counts())
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
print(df2)

Generamos un pequeño data set con las intancias que tenga OWID.

In [ ]:
dataset_OWID=dataset2[dataset2["iso_code"].str.contains("OWID")]
sum_OWID_NCases=dataset_OWID["new_cases"].sum()
sum_OWID_NCases

Vemos cuales son los OWID que tenemos que eliminar.

In [ ]:
print(dataset_OWID.shape)
dataset_OWID["iso_code"].unique()

Generamos un dataset que no contengan estos OWID, este data set es en el que vamos a trabajar.

In [ ]:
dataset_not_OWID=dataset2.set_index("iso_code")

In [ ]:
dataset_not_OWID=dataset_not_OWID.drop(['OWID_AFR', 'OWID_ASI', 'OWID_EUR', 'OWID_EUN', 'OWID_HIC',
       'OWID_INT', 'OWID_KOS', 'OWID_LIC', 'OWID_LMC', 'OWID_NAM',
       'OWID_CYN', 'OWID_OCE', 'OWID_SAM'])

In [ ]:
dataset_not_OWID.reset_index(inplace=True)

In [ ]:
print(dataset_not_OWID.shape)
dataset_not_OWID.head()

In [ ]:
sum_not_OWID_NCases=dataset_not_OWID["new_cases"].sum()
sum_not_OWID_NCases

Corroboramos que no quede ninguno

In [ ]:
dataset_not_OWID["iso_code"].unique()

Volvamos a revisar la forma que tendrá este dataset

In [ ]:
dataset_not_OWID.shape

Al haber tanta informacion, procedemos a eliminar las columnas que no vamos a utilizar para poder reducir la dimensionalidad que presenta la base de datos.

In [ ]:
columnas_descartar= ['excess_mortality_cumulative_absolute', 'excess_mortality_cumulative',
                     'excess_mortality', 'excess_mortality_cumulative_per_million', 'human_development_index', 'life_expectancy' ,     
                     'female_smokers', 'male_smokers','handwashing_facilities','diabetes_prevalence', 'cardiovasc_death_rate', 'gdp_per_capita',
                     'extreme_poverty','aged_65_older','aged_70_older', 'new_people_vaccinated_smoothed',
                     'new_people_vaccinated_smoothed_per_hundred', 'new_vaccinations_smoothed_per_million','iso_code', 'continent',
                     'total_boosters', 'total_boosters_per_hundred','new_vaccinations_smoothed', 'total_vaccinations_per_hundred',
                     'total_vaccinations','tests_per_case', 'tests_units','new_tests_smoothed', 'new_tests_smoothed_per_thousand',
                     'new_deaths_smoothed','new_cases_smoothed','new_cases_smoothed_per_million','new_deaths_smoothed_per_million',
                     'weekly_icu_admissions','weekly_icu_admissions_per_million', 'weekly_hosp_admissions','weekly_hosp_admissions_per_million',
                     "total_tests_per_thousand","new_tests_per_thousand","people_vaccinated_per_hundred",
                     "people_fully_vaccinated_per_hundred","median_age","new_tests","total_tests","people_fully_vaccinated","new_vaccinations","total_deaths",
                     "new_deaths_per_million","total_deaths_per_million","new_cases_per_million","total_cases_per_million"
                     , "icu_patients_per_million","total_deaths","hosp_patients","positive_rate"]
                    
dataset_covid  =dataset_not_OWID.drop(columns=columnas_descartar)
print(dataset_covid.shape)
dataset_covid.head()

Normalizamos las columnas que utilizaremos llevandolas a la escala de 100 mil por habitantes.

In [ ]:
def _population(fila):
    resultado=(fila["population"]/100000)
    return resultado

In [ ]:
dataset_covid["population_per_100Thousand"]=dataset_covid.apply( _population,axis=1)

In [ ]:
dataset_covid["total_cases_per_100Thousand"]=dataset_covid.apply(lambda row: row["total_cases"]/row['population']* 100000, axis=1)
dataset_covid["new_cases_per_100Thousand"]=dataset_covid.apply(lambda row: row["new_cases"]/row['population']* 100000, axis=1)
dataset_covid["new_deaths_per_100Thousand"]=dataset_covid.apply(lambda row: row["new_deaths"]/row['population']* 100000, axis=1)
dataset_covid["population_density_per_100Thousand"]=dataset_covid.apply(lambda row: row["population_density"]/100000, axis=1)

Luego de convertidas, eliminamos las columnas originales.

In [ ]:
columnas_Descartar= ["population","total_cases","new_cases","new_deaths","population_density"]
dataset_covid =dataset_covid.drop(columns=columnas_Descartar)
print(dataset_covid.shape)
dataset_covid.head()

Observamos que hay muchas casos faltantes pero por ahora no nos vamos a ocupar.

In [ ]:
for col in dataset_covid.columns:
    nulos=dataset_covid[col].isna().sum()
    print(col, "{:.2f}%".format(nulos*100/125607))

Lo primero que haremos es observar la curva de contagios de 10 paises del norte, que es donde mas temprano de propago el virus.




## Gráficos

Lo primero que haremos es observar la curva de contagios de 10 paises del norte, que es donde más temprano de propagó el virus.

In [ ]:
paises=dataset_covid['location'] #cargo el vector de paises
confirmados=dataset_covid['total_cases_per_100Thousand'] #carga vector todos los casos confirmados

In [ ]:
lista_paises_norte=['Denmark','Sweden','United States','France','Canada','Germany','Finland','United Kingdom','Spain','Italy']

In [ ]:
for i in lista_paises_norte:
    ind_=np.where(paises==i)
    ind_=ind_[0]
    casos_=confirmados[ind_]
    dias=np.arange(0,len(casos_))
    plt.plot(dias,casos_,label=i)
    plt.title("Casos de covid-19 por pais")
    plt.xlabel('dias')
    plt.ylabel('cantidad de casos_per_100Thousand')    
    plt.grid()
    plt.legend()

Estas son las curvas de *casos totales por 100K habitantes* de los 10 paises seleccionados. Aquí no se ven las olas, ya que estas se ven en la curva de casos confirmados por día, es decir, de casos nuevos. Esta habla de los acumulados, entonces siempre se agregan.

Para ver mejor la curva, podemos tomar el logartimo de los datos. Una curva exponencial se ve lineal en escala logarítmica

In [ ]:
for i in lista_paises_norte:
    ind_=np.where(paises==i)
    ind_=ind_[0]
    casos_=confirmados[ind_]
    dias=np.arange(0,len(casos_))
    plt.plot(dias,np.log(casos_),label=i)
    plt.title("Casos de covid-19 por pais-log")
    plt.xlabel('dias')
    plt.ylabel('log cantidad de casos_per_100Thousand')
    plt.grid()
    plt.legend()

Veamos lo que pasa en los primeros 90 días desde que se detectó el primer caso en estos paises, para ver cómo fue el  inicial.

In [ ]:
for i in lista_paises_norte:
    ind_=np.where(paises==i)
    ind_=ind_[0]
    casos_=confirmados[ind_]
    casos_principio=casos_[0:90]
    dias_principio=dias[0:90] 
    plt.plot(dias_principio,casos_principio,label=i)
    plt.title("Casos de covid-19 por pais- dias 0 a 90")
    plt.xlabel('dias')
    plt.ylabel('cantidad de casos_per_100Thousand')
    plt.grid()
    plt.legend()

Es difícil determinar si estas partes son exponenciales. Analicemos qué pasa en escala logarítmica.

In [ ]:
for i in lista_paises_norte:
    ind_=np.where(paises==i)
    ind_=ind_[0]
    casos_=confirmados[ind_]
    casos_principio=casos_[0:90]
    dias_principio=dias[0:90]
    plt.plot(dias_principio,np.log(casos_principio),label=i)
    plt.title("Casos de covid-19 por pais-Logaritmica- dias 0 a 90")
    plt.xlabel('dias')
    plt.ylabel('log cantidad de casos_per_100Thousand')
    plt.grid()
    plt.legend()

Tomaremos diferentes periodos de tiempo para diferentes paises ya que ese fue el comportamiento más sostenido se da en tiempos diferentes en muchos de ellos.

In [ ]:
lista_paises_norteA=['Denmark','Sweden','Canada','Germany','Finland','United Kingdom','Spain']
for i in lista_paises_norteA:
    ind_=np.where(paises==i)
    ind_=ind_[0]
    casos_=confirmados[ind_]
    casos_principio=casos_[25:71]
    dias_principio=dias[25:71]
    plt.plot(dias_principio,casos_principio,label=i)
    plt.title("Casos de covid-19 por pais- dias 25 a 71")
    plt.xlabel('dias')
    plt.ylabel('cantidad de casos_per_100Thousand')
    plt.grid()
    plt.legend()


In [ ]:
ind_=np.where(paises=="France")
ind_=ind_[0]
casos_=confirmados[ind_]
casos_principio=casos_[30:71]
dias_principio=dias[30:71]
plt.plot(dias_principio,casos_principio,color="r",label="France")
plt.title("Casos de covid-19 por pais- dias 30 a 71")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_por_millon')
plt.grid()
plt.legend()

In [ ]:
ind_=np.where(paises=='Italy')
ind_=ind_[0]
casos_=confirmados[ind_]
casos_principio=casos_[20:71]
dias_principio=dias[20:71]
plt.plot(dias_principio,casos_principio,color="skyblue",label="Italy")
plt.title("Casos de covid-19 por pais- dias 20 a 71")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_por_millon')
plt.grid()
plt.legend()

In [ ]:
ind_=np.where(paises=='United States')
ind_=ind_[0]
casos_=confirmados[ind_]
casos_principio=casos_[40:71]
dias_principio=dias[40:71]
plt.plot(dias_principio,casos_principio,color="grey",label='United States')
plt.title("Casos de covid-19 por pais- dias 40 a 71")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_por_millon')
plt.grid()
plt.legend()

Observando los cuatro graficos Logaritmicos notamos que los periodos seleccionados para los diferentes paises estan correctos, ya que se puede observar que las curvas exponenciales llevadas a logaritmica se asemejan a lineales y esto se puede notar con facilidad en los graficos.

In [ ]:
for i in lista_paises_norteA:
    ind_=np.where(paises==i)
    ind_=ind_[0]
    casos_=confirmados[ind_]
    casos_principio=casos_[25:71]
    dias_principio=dias[25:71]
    plt.plot(dias_principio,np.log(casos_principio),label=i)
    plt.title("Casos de covid-19 por pais-LOGARITMICA- dias 25 a 71")
    plt.xlabel('dias')
    plt.ylabel('log cantidad de casos_per_100Thousand')
    plt.grid()
    plt.legend()

In [ ]:
ind_=np.where(paises=="France")
ind_=ind_[0]
casos_=confirmados[ind_]
casos_principio=casos_[40:71]
dias_principio=dias[40:71]
plt.plot(dias_principio,np.log(casos_principio),color="r",label="France")
plt.title("Casos de covid-19 por pais-LOGARITMICA- dias 40 a 71")
plt.xlabel('dias')
plt.ylabel('log cantidad de casos_por_millon')
plt.grid()
plt.legend()

In [ ]:
ind_=np.where(paises=='Italy')
ind_=ind_[0]
casos_=confirmados[ind_]
casos_principio=casos_[40:71]
dias_principio=dias[40:71]
plt.plot(dias_principio,np.log(casos_principio),color="skyblue",label="Italy")
plt.title("Casos de covid-19 por pais-LOGARITMICA- dias 40 a 71")
plt.xlabel('dias')
plt.ylabel('log cantidad de casos_por_millon')
plt.grid()
plt.legend()

In [ ]:
ind_=np.where(paises=='United States')
ind_=ind_[0]
casos_=confirmados[ind_]
casos_principio=casos_[40:71]
dias_principio=dias[40:71]
plt.plot(dias_principio,np.log(casos_principio),color="grey",label='United States')
plt.title("Casos de covid-19 por pais-LOGARITMICA- dias 40 a 71")
plt.xlabel('dias')
plt.ylabel('log cantidad de casos_por_millon')
plt.grid()
plt.legend()

## ¿Cuál es el k de cada país?

Averiguaremos el  utilizando un ajuste hecho con el método de cuadrados mínimos, ajustando los datos al modelo .



Confirmados = e^{k(\t - \t_0)}

In [ ]:
#definimos función para ajustar modelo
def exponencial(t,k,t0):
  C=np.exp(k*(t-t0))
  return C

In [ ]:
ind_=np.where(paises=='United States')
ind_=ind_[0]
casos_=confirmados[ind_]
casos_principioUS=casos_[40:71]
dias_principioUS=dias[40:71]
#ajustamos
popt, pcov = curve_fit(exponencial, dias_principioUS, casos_principioUS)
popt

In [ ]:
kUS=popt[0]
t0=popt[1]
casos_simuUS=exponencial(dias_principioUS,kUS,t0)
plt.plot(dias_principioUS,casos_principioUS,'.')
plt.plot(dias_principioUS,casos_simuUS)
plt.title("Casos de 'United States'.SUMULADO VS REAL")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_per_100Thousand')

El ajuste es muy bueno.

In [ ]:
ind_=np.where(paises=='France')
ind_=ind_[0]
casos_=confirmados[ind_]
casos_principioFR=casos_[30:71]
dias_principioFR=dias[30:71]
#ajustamos
popt, pcov = curve_fit(exponencial, dias_principioFR, casos_principioFR)
popt

In [ ]:
kFR=popt[0]
t0=popt[1]
casos_simuFR=exponencial(dias_principioFR,kFR,t0)
plt.plot(dias_principioFR,casos_principioFR,'.')
plt.plot(dias_principioFR,casos_simuFR)
plt.title("Casos de 'France'.SUMULADO VS REAL")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_per_100Thousand')

In [ ]:
ind_=np.where(paises=='Canada')
ind_=ind_[0]
casos_=confirmados[ind_]
casos_principioCA=casos_[25:71]
dias_principioCA=dias[25:71]
#ajustamos
popt, pcov = curve_fit(exponencial, dias_principioCA, casos_principioCA)
popt

In [ ]:
kCA=popt[0]
t0=popt[1]
casos_simuCA=exponencial(dias_principioCA,kCA,t0)
plt.plot(dias_principioCA,casos_principioCA,'.')
plt.plot(dias_principioCA,casos_simuCA)
plt.title("Casos de 'Canada'.SUMULADO VS REAL")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_per_100Thousand')

In [ ]:
ind_=np.where(paises=='Germany')
ind_=ind_[0]
casos_=confirmados[ind_]
casos_principioGR=casos_[25:71]
dias_principioGR=dias[25:71]
#ajustamos
popt, pcov = curve_fit(exponencial, dias_principioGR, casos_principioGR)
popt

In [ ]:
kGE=popt[0]
t0=popt[1]
casos_simuGR=exponencial(dias_principioGR,kGE,t0)
plt.plot(dias_principioGR,casos_principioGR,'.')
plt.plot(dias_principioGR,casos_simuGR)
plt.title("Casos 'Germany'.SUMULADO VS REAL")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_per_100Thousand')

In [ ]:
ind_=np.where(paises=='Finland')
ind_=ind_[0]
casos_=confirmados[ind_]
casos_principioFI=casos_[25:71]
dias_principioFI=dias[25:71]
#ajustamos
popt, pcov = curve_fit(exponencial, dias_principioFI, casos_principioFI)
popt

In [ ]:
kFI=popt[0]
t0=popt[1]
casos_simuFI=exponencial(dias_principioFI,kFI,t0)
plt.plot(dias_principioFI,casos_principioFI,'.')
plt.plot(dias_principioFI,casos_simuFI)
plt.title("Casos de 'Finland'.SUMULADO VS REAL")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_per_100Thousand')

In [ ]:
ind_=np.where(paises=='United Kingdom')
ind_=ind_[0]
casos_=confirmados[ind_]
casos_principioUK=casos_[25:71]
dias_principioUK=dias[25:71]
#ajustamos
popt, pcov = curve_fit(exponencial, dias_principioUK, casos_principioUK)
popt

In [ ]:
kUK=popt[0]
t0=popt[1]
casos_simuUK=exponencial(dias_principioUK,kUK,t0)
plt.plot(dias_principioUK,casos_principioUK,'.')
plt.plot(dias_principioUK,casos_simuUK)
plt.title("Casos de 'United Kingdom'.SUMULADO VS REAL")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_per_100Thousand')

In [ ]:
ind_=np.where(paises=='Spain')
ind_=ind_[0]
casos_=confirmados[ind_]
casos_principioSP=casos_[25:71]
dias_principioSP=dias[25:71]
#ajustamos
popt, pcov = curve_fit(exponencial, dias_principioSP, casos_principioSP)
popt

In [ ]:
kSP=popt[0]
t0=popt[1]
casos_simuSP=exponencial(dias_principioSP,kSP,t0)
plt.plot(dias_principioSP,casos_principioSP,'.')
plt.plot(dias_principioSP,casos_simuSP)
plt.title("Casos de 'Spain'.SUMULADO VS REAL")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_per_100Thousand')

In [ ]:
ind_=np.where(paises=='Sweden')
ind_=ind_[0]
casos_=confirmados[ind_]
casos_principioSW=casos_[25:71]
dias_principioSW=dias[25:71]
#ajustamos
popt, pcov = curve_fit(exponencial, dias_principioSW, casos_principioSW)
popt

In [ ]:
kSW=popt[0]
t0=popt[1]
casos_simuSW=exponencial(dias_principioSW,kSW,t0)
plt.plot(dias_principioSW,casos_principioSW,'.')
plt.plot(dias_principioSW,casos_simuSW)
plt.title("Casos de 'Sweden'.SUMULADO VS REAL")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_per_100Thousand')

Compararemos y analisaremos el K de los 10 paises seleccionados y el K del mundo. Veremos si se puede decir que el comportamiento de los paises seleccionados es igual al del mundo.

In [ ]:
list_total_paises=paises.unique().tolist()

In [ ]:
for i in list_total_paises:
    ind_=np.where(paises==i)
    ind_=ind_[0]
    casos_=confirmados[ind_]
    casos_principioW=casos_[30:71]
    dias_principioW=dias[30:71]
    
    
for i in lista_paises_norte:
    ind_=np.where(paises==i)
    ind_=ind_[0]
    casos_=confirmados[ind_]
    casos_principio10pn=casos_[30:71]
    dias_principio10pn=dias[30:71]

    
popt, pcov = curve_fit(exponencial, dias_principio10pn, casos_principio10pn)
print(popt)       

plt.figure(figsize=(18,9))
plt.suptitle("Graficos comparativos de casos Covid-19. Mundo vs 10 paises Norte")
plt.subplot(2,2,1)
plt.plot(dias_principioW,casos_principioW,color="r")
plt.title("Casos de covid-19 Mundo")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_per_100Thousand')    
plt.grid()
plt.subplot(2,2,2)
plt.plot(dias_principioW,np.log(casos_principioW),color="r")
plt.title("Casos de covid-19 Mundo -LOG-")
plt.xlabel('dias')
plt.ylabel('log cantidad de casos_per_100Thousand')
plt.grid()
plt.subplot(2,2,3)
plt.plot(dias_principio10pn,casos_principio10pn)
plt.title("Casos de covid-19 de 10 paises del Norte")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_per_100Thousand')    
plt.grid()
plt.subplot(2,2,4)
plt.plot(dias_principio10pn,np.log(casos_principio10pn))
plt.title("Casos de covid-19 de 10 paises del Norte -LOG-")
plt.xlabel('dias')
plt.ylabel('log cantidad de casos_per_100Thousand')
plt.grid()
plt.show()

Tomamos un promedio de los dias de inicio de la curva de casos de los paises seleccionados, dando el dia 30 hasta el dia 71 donde en todos los paises seleccionados se estabiliza la curva de casos.

Desde ya se nota que no es comparable por que vemos que en el caso de los paises se nota la exponencialidad en el periodo hasta la estabilizacion, pero con respecto al mundo se nota que en conjunto ni siquiera empezo esta exponencialidad, esta mucho mas retrazado, esto es logico que la pandemia fue expandiendose por todos los paises del mundo a traves del tiempo, que indudablemente no fue en el mismo periodo que los 10 paises del norte seleccionados.

In [ ]:
k=popt[0]
t0=popt[1]
casos_simuMundo=exponencial(dias_principioW,k,t0)
casos_simu10pn=exponencial(dias_principioW,k,t0)

plt.figure(figsize=(10,5))
plt.suptitle("Graficos comparativos de casos Covid-19. Mundo vs 10 paises Norte.SUMULADO VS REAL")
plt.subplot(1,2,1)
plt.plot(dias_principio10pn,casos_principio10pn,'.',label="Real")
plt.plot(dias_principio10pn,casos_simu10pn,label="Simulado")
plt.title("Casos de 10 paises del Norte.SIMULADO VS REAL")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_per_100Thousand')
plt.grid()
plt.subplot(1,2,2)
plt.title("Casos del Mundo.SIMULADO VS REAL")
plt.plot(dias_principioW,casos_principioW,'.',label="Real")
plt.plot(dias_principioW,casos_simuMundo,label="Simulado")
plt.xlabel('dias')
plt.ylabel('cantidad de casos_per_100Thousand')
plt.grid()

Volvemos a notar que los paises seleccionados con respecto al mundo no es comparable ni extrapolable.

Aqui tomamos la K de los paises seleccionados para simular la exponencialidad de los casos de mundo y vemos que en el caso de los 10 paise se acomodan bastante bien con los casos reales, pero en el grafico del mundo lejos esta de poder predecir los casos del mundo con el K de los 10 paises; cuales son los motivos los mismo que los que fueren explicados en el grafico anterior, el tiempo en que de expandio el virus en estos paises y el resto del mundo.

In [ ]:
k10p = [kSW,kUS,kFR,kCA,kGE,kFI,kUK,kSP]
k10p = pd.DataFrame(k10p)

In [ ]:
k10p.describe()

In [ ]:
mu_k10p=np.mean(k10p)
sigma_k10p=np.std(k10p)
z=1.96
n100=len(k10p)
int_conf= [mu_k10p-z*sigma_k10p/np.sqrt(n100),mu_k10p+z*sigma_k10p/np.sqrt(n100)]
print(int_conf )
print((int_conf[1] - int_conf[0]))

Vemos entonces que el intervalo de confianza de confianza de 𝑘 para los 10 paises escogidos se encuentra entre [0.89418 , 0.134245] con una diferencia de 0.044827 entre los valores del intervalo.

## Elección de la política pública

Trabajaremos de aquí en adelante con los países que decidieron realizar una cuarentena obligatoria vis-a-vis los países que decidieron no realizar la cuarentena obligatoria o adoptaron el enfoque de cuarentena inteligente, el cuál consiste de un aislamiento mucho más holgado en dónde sólo las personas con síntomas son aconsejadas para quedarse en sus hogares.

La elección de este tipo de política responde a que fue una de los primeros enfoques aplicaods por los países del mundo como forma de combate ante la Pandemia restringiendo la movilidad de las poblaciones y generando los primeros indicadores para medir el avance del COVID-19 en el mundo.

Por otro lado, se decidió usar este política con respecto a otras formas de políticas públicas tales como el incremento del gasto público con respecto al PIB, indicadores de riesgo país o programas de transferencias estatales dada la simplicidad y efectividad que ha tenido el enfoque de las cuarentenas y su facilidad de comparabilidad entre países con respecto al enfoque de utilizar medidas ecónomicas en dónde hay que tener en cuenta efectos como la paridad cruzada de los tipos de cambio, la inflación y el crecimiento ecónomico y factores socio-culturales,

Con lo cual, seleccionaremos los 10 paises según la Politica elegida para el analisis "Cuarentena Nacional Obligatoria" vs "Cuarentena Regional/No Cuarentena/Cuarentena Inteligente".



Para seleccionar que paises hicieron cuarentena y cuales no; y en que fechas empezeron, esta informacion la obtuvimos del siguiente link.

https://es.wikipedia.org/wiki/Confinamiento_por_la_pandemia_de_COVID-19#Cuarentena_por_pa%C3%ADses

CUARENTENA REGIONAL/NO HICIERON CUARENTENA/ CUARENTENA INTELIGENTE.-Binario 1

'United States'-19/03/2020 AL Inicio Vacunacion.

'Finland'-------27/03/2020 AL Inicio Vacunacion.

'Germany'-------20/03/2020 AL Inicio Vacunacion.

'Sweden'--------19/03/2020 AL Inicio Vacunacion.

'Canada'--------19/03/2020 AL Inicio Vacunacion.

CUARENTENA NACIONAL.-Binario 0

'Spain'-----------15/03/2020 AL Inicio Vacunacion.

'Italy'-----------09/03/2020 AL Inicio Vacunacion.

'Denmark'---------11/03/2020 AL Inicio Vacunacion.

'France'----------17/03/2020 AL Inicio Vacunacion.

'United Kingdom'--20/03/2020 AL Inicio Vacunacion.

El motivo por el cual dejamos la variable "people_vaccinated" era para determinar la fecha de inicio de la vacunacion y poder aislar la politica de cuarentena.

In [ ]:
dataset_covid.describe()

Tomamos los 10 paises, 5 de los cuales hicieron "Cuarentena Nacional Obligarotia" y 5 de los cuales hicieron "Cuarentena Regional o No hicieron Cuarentena", tambien tomamos 3 paises mas para luego corroborar si el mejor modelo segun lo indicadores de la curvas logra predecir si estos paises hicieron o no cuarentena nacional.

In [ ]:
dataset_paises_Analisis=dataset_covid[dataset_covid.location.isin(['United States','Finland','Germany','Sweden','Canada','Spain','Italy','Denmark','France','United Kingdom',"Brazil",'Argentina',"Croatia" ])]


In [ ]:
dataset_paises_Analisis["date"]=pd.to_datetime(dataset_paises_Analisis.date)

In [ ]:
dataset_paises_Analisis=dataset_paises_Analisis.set_index("date")

In [ ]:
DS_US_all_date=dataset_paises_Analisis[(dataset_paises_Analisis["location"]=='United States')]
DS_FI_all_date=dataset_paises_Analisis[(dataset_paises_Analisis["location"]=='Finland')]
DS_GE_all_date=dataset_paises_Analisis[(dataset_paises_Analisis["location"]=='Germany')]
DS_SW_all_date=dataset_paises_Analisis[(dataset_paises_Analisis["location"]=='Sweden')]
DS_CA_all_date=dataset_paises_Analisis[(dataset_paises_Analisis["location"]=='Canada')]
DS_SP_all_date=dataset_paises_Analisis[(dataset_paises_Analisis["location"]=='Spain')]
DS_IT_all_date=dataset_paises_Analisis[(dataset_paises_Analisis["location"]=='Italy')]
DS_DE_all_date=dataset_paises_Analisis[(dataset_paises_Analisis["location"]=='Denmark')]
DS_FR_all_date=dataset_paises_Analisis[(dataset_paises_Analisis["location"]=='France')]
DS_UK_all_date=dataset_paises_Analisis[(dataset_paises_Analisis["location"]=='United Kingdom')]
DS_BR_all_date=dataset_paises_Analisis[(dataset_paises_Analisis["location"]=='Brazil')]
DS_AR_all_date=dataset_paises_Analisis[(dataset_paises_Analisis["location"]=='Argentina')]
DS_CR_all_date=dataset_paises_Analisis[(dataset_paises_Analisis["location"]=='Croatia')]

Seleccionamos las fechas de duración de las cuarentenas de los diferentes paises.

In [ ]:
DS_US=DS_US_all_date['2020-03-19':'2020-12-12']
DS_FI=DS_FI_all_date['2020-03-27':'2021-01-24']
DS_GE=DS_GE_all_date['2020-03-20':'2020-12-26']
DS_SW=DS_SW_all_date['2020-03-19':'2020-12-26']
DS_CA=DS_CA_all_date['2020-03-19':'2020-12-13']
DS_SP=DS_SP_all_date["2020-03-15":"2021-01-03"]
DS_IT=DS_IT_all_date['2020-03-09':'2020-12-26']
DS_DE=DS_DE_all_date['2020-03-11':'2020-11-30']
DS_FR=DS_FR_all_date['2020-05-17':'2020-12-26']
DS_UK=DS_UK_all_date['2020-07-20':'2021-01-09'] 
DS_BR=DS_BR_all_date['2020-03-19':'2020-12-13']
DS_AR=DS_AR_all_date['2020-03-20':'2020-12-13']
DS_CR=DS_CR_all_date['2020-03-18':'2020-12-13']

In [ ]:
DSet_paises_Analisis=DS_US.append(DS_FI)
DSet_paises_Analisis=DSet_paises_Analisis.append(DS_GE)
DSet_paises_Analisis=DSet_paises_Analisis.append(DS_SW)
DSet_paises_Analisis=DSet_paises_Analisis.append(DS_CA)
DSet_paises_Analisis=DSet_paises_Analisis.append(DS_SP)
DSet_paises_Analisis=DSet_paises_Analisis.append(DS_IT)
DSet_paises_Analisis=DSet_paises_Analisis.append(DS_DE)
DSet_paises_Analisis=DSet_paises_Analisis.append(DS_FR)
DSet_paises_Analisis=DSet_paises_Analisis.append(DS_UK)
DSet_paises_Analisis=DSet_paises_Analisis.append(DS_BR)
DSet_paises_Analisis=DSet_paises_Analisis.append(DS_AR)
DSet_paises_Analisis=DSet_paises_Analisis.append(DS_CR)

In [ ]:
DSet_paises_Analisis.reset_index(inplace=True)

In [ ]:
DSet_paises_Analisis["new_deaths_per_100Thousand"].fillna(0,inplace=True)
DSet_paises_Analisis["hosp_patients_per_million"].fillna(0,inplace=True)
DSet_paises_Analisis["stringency_index"].fillna(0,inplace=True) 

In [ ]:
DSet_paises_Analisis.isna().sum()

In [ ]:
DSet_paises_Analisis=DSet_paises_Analisis[(DSet_paises_Analisis["hosp_patients_per_million"]>=0) ]
DSet_paises_Analisis=DSet_paises_Analisis[(DSet_paises_Analisis["new_deaths_per_100Thousand"]>=0) ]
DSet_paises_Analisis=DSet_paises_Analisis[(DSet_paises_Analisis["stringency_index"]>=0) ]

In [ ]:
dataset_cuarentenaRegion=DSet_paises_Analisis[DSet_paises_Analisis.location.isin(['United States','Finland','Germany','Sweden','Canada'])]
dataset_cuarentenaRegion["si_no_Cuarentena"]=1
dataset_cuarentenaNacion=DSet_paises_Analisis[DSet_paises_Analisis.location.isin(['Spain','Italy','Denmark','France','United Kingdom'])]
dataset_cuarentenaNacion["si_no_Cuarentena"]=0

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from scipy.special import expit
from sklearn.naive_bayes import GaussianNB

## Evaluando estrategias

Evaluaremos las estrategias de los paises a traves de valores objetivos tomados de distintas fuentes, en titulos explicitaremos que indicador estamos evaluando.



Capacidad de utlizacion de las UCI

De acuerdo al sitio COVID Act Now, una plataforma creada por las universidades de Harvard, Stanford y Georgetown destinado al trackeo de la pandemia en los Estados Unidos, una de las métricas claves para poder evaluar el éxito de la política sanitaria de un país es la capacidad de utilización de las unidades de terapia intensiva. En donde dicha variable proviene del uso de las camas de terapia intensiva.

De acuerdo a esta plataforma, un nivel de utilización de las camas superior al 80% se toma como crítico.


Link de las métricas utilizadas por esta plataforma: 
https://covidactnow.org/covid-risk-levels-metrics#icu-capacity-used  




In [ ]:
def _icu(fila):
    resultado=(fila["icu_patients"]/1000)
    return resultado

In [ ]:
def rango_Aumen_Baja (fila): 
  if fila > 0.8 :
     return 0
  else : 
     return 1 

Evaluamos la efectividad de la Cuarentena Regional.

In [ ]:
dataset_cuarentenaRegion["icu_patients_per_thousand"]=dataset_cuarentenaRegion.apply(_icu,axis=1)

In [ ]:
dataset_cuarentenaRegion["icu_capacity"] = (dataset_cuarentenaRegion["icu_patients_per_thousand"] / dataset_cuarentenaRegion["hospital_beds_per_thousand"])*100

In [ ]:
dataset_cuarentenaRegion['icu_capacity'] = dataset_cuarentenaRegion['icu_capacity'].fillna(0)

In [ ]:
dataset_cuarentenaRegion["Aument_BajaNC"] = dataset_cuarentenaRegion["icu_capacity"].map(rango_Aumen_Baja)


In [ ]:
dataset_cuarentenaRegion.head()

In [ ]:
sns.countplot(x="Aument_BajaNC",data=dataset_cuarentenaRegion)
plt.show()

Vemos que la cuarentena regional fue efectiva, lo vemos en el grafico donde la categoria cero (0) que significa que la capacidad de las unidades de terapia intensiva es muy superior a la categoria uno (1) que es la que afirma que mejoró. La información provista indica que la no utilización generaría una mayor ocupación de las camas de terapia intensiva

Evaluamos la efectividad de la Cuarentena Nacional Obligatoria.

In [ ]:
dataset_cuarentenaNacion["icu_patients_per_thousand"]=dataset_cuarentenaNacion.apply(_icu,axis=1)

In [ ]:
dataset_cuarentenaNacion["icu_capacity"] = (dataset_cuarentenaNacion["icu_patients_per_thousand"] / dataset_cuarentenaNacion["hospital_beds_per_thousand"])*100

In [ ]:
dataset_cuarentenaNacion['icu_capacity'] = dataset_cuarentenaNacion['icu_capacity'].fillna(0)

In [ ]:
dataset_cuarentenaNacion["Aument_BajaNC"] = dataset_cuarentenaNacion["icu_capacity"].map(rango_Aumen_Baja)


In [ ]:
dataset_cuarentenaNacion["Aument_BajaNC"] = dataset_cuarentenaNacion["icu_capacity"].map(rango_Aumen_Baja)

In [ ]:
dataset_cuarentenaNacion.head(1)

In [ ]:
sns.countplot(x="Aument_BajaNC",data=dataset_cuarentenaNacion)
plt.show()

Se puede concluir que, tanto con cuarentena obligatorio como sin ella, la capacidad de uso de las unidades de terapia intensiva fueron altas en ambos casos. Sin embargo, cabe resaltar que el acumulado es mucho mayor en el caso de la cuarentena inteligente, lo cual muestra cierto éxito del enfoque de la cuarentena obligatoria.

Tasa de positividad

De acuerdo a la plataforma previamente mencionada, la tasa de positivdad es otros de los indicadores que se utilizan.

De acuerdo a ellos, un nivel de positividad superior al 20% diario es indicativo de que la estrategia encarada no es muy buena.




In [ ]:
def rango_Aumen_Bajatr (fila): 
  if fila > 0.2 :
     return 0
  else : 
     return 1 

In [ ]:
dataset_cuarentenaRegion["positive test rate"] = (dataset_cuarentenaRegion["new_cases_per_100Thousand"] / dataset_cuarentenaRegion["population_per_100Thousand"])*100

In [ ]:
dataset_cuarentenaRegion.head()

In [ ]:
dataset_cuarentenaRegion["Aument_Bajatr"] = dataset_cuarentenaRegion["positive test rate"].map(rango_Aumen_Bajatr)

In [ ]:
sns.countplot(x="Aument_Bajatr",data=dataset_cuarentenaRegion)
plt.show()

Veamos el éxito de la cuarentena obligatoria

In [ ]:
dataset_cuarentenaNacion["positive test rate"] = (dataset_cuarentenaNacion["new_cases_per_100Thousand"] / dataset_cuarentenaNacion["population_per_100Thousand"])*100

In [ ]:
dataset_cuarentenaNacion.head()

In [ ]:
dataset_cuarentenaNacion["Aument_Bajatr"] = dataset_cuarentenaNacion["positive test rate"].map(rango_Aumen_Bajatr)

In [ ]:
sns.countplot(x="Aument_Bajatr",data=dataset_cuarentenaNacion)
plt.show()

Notamos que segun este indicador las distintas cuarentenas no fueron sufucientemente efectivas, pero que algun resultado tuviernon.

Nuevamente, cabe resaltar que el acumulado es mucho mayor en el caso de la cuarentena inteligente, lo cual muestra cierto éxito del enfoque de la cuarentena obligatoria.

Tasa de reproducción

Del siguiente link de la BBC tomamos la informacion que nos hace entender como funciona el ratio de reproduccion el cual si este es menor a 1 es que se esta logrando frenar el contagio, si es mayor a uno este sigue reproduciendoce.El valor inicia de la pandemia fue de 2,5.

Este a mi entender es el mejor indicador para medir la eficacia de cualquier Politica sanitaria de cualquier pais, en este proyecto esta politica es la cuarentena Nacional vs. la cuarentena regional o no cuarentena.

https://www.bbc.com/mundo/noticias-51469198

In [ ]:
def rango_Aumen_BajaRR (fila): 
  if fila < 1 :
     return 0
  else : 
     return 1 

In [ ]:
dataset_cuarentenaRegion["Aument_BajaRR"] = dataset_cuarentenaRegion["reproduction_rate"].map(rango_Aumen_BajaRR)

In [ ]:
dataset_cuarentenaRegion.head(1)   

In [ ]:
sns.countplot(x="Aument_BajaRR",data=dataset_cuarentenaRegion)
plt.show()

Vemos que con este criterio de evaluacion los paises que hicieron cuarentena regional no fue muy eficiente ya que en la mayoria de los casos no se pudo frenar el contagio.



Evaluamos la efectividad de la Cuarentena Nacional Obligatoria.

  

In [ ]:
dataset_cuarentenaNacion["Aument_BajaRR"] = dataset_cuarentenaNacion["reproduction_rate"].map(rango_Aumen_BajaRR)


In [ ]:
dataset_cuarentenaNacion.head(1)


In [ ]:
sns.countplot(x="Aument_BajaRR",data=dataset_cuarentenaNacion)
plt.show()

Notamos que segun este indicador las distintas cuarentenas no fueron sufucientemente efectivas, pero que algun resultado tuviernon.

Por todo lo visto con estos 3 indicadores, la cuarentena obligatoria ha tenido una mejor performance en la lucha contra el COVID-19 aunque en cualquiera de los casos ninguno de los dos enfoques ha sido lo suficientemente eficaz para detener el avance de la pandemia.

**Perparando los datos para modelar**

Unimos los dos dataset de paises que hicieron cuarentena nacional y regional.



In [ ]:
DS_paises_Analisis=dataset_cuarentenaNacion.append(dataset_cuarentenaRegion)
print(DS_paises_Analisis.shape)

Corroboramos que no haya faltantes, y si que en "people vaccinated este 100% faltantes lo que nos indicaria que aislamos bien la politica de cuarentena.

Vemos que en este dataset estan las variables Aument_BajaNC / Aument_BajaRR / Aument_Bajatr que son la clasificaciones que hicimos anteriormente para evaluar si los distintos estilo de cuarentena fueron o no eficaces, estas variables son las que vamos a poner en los modelos de clasificacion (con otras) para ver si es posible predecir si un pais hizo o no cuarentena nacional obligatoria.

In [ ]:
DS_paises_Analisis = DS_paises_Analisis.fillna(0)

In [ ]:
for col in DS_paises_Analisis.columns:
    nulos=DS_paises_Analisis[col].isna().sum()
    print(col, "{:.2f}%".format(nulos*100/2630))

In [ ]:
print(DS_paises_Analisis.shape)
DS_paises_Analisis.describe()

In [ ]:
sns.countplot(x="si_no_Cuarentena",data=DS_paises_Analisis)
plt.show()

## Modelando

Para el modelado vamos a utilizar la Regresion logistica que es un clasificador y la vamos a evaluar a traves del accuracy, haremos tres modelos tomando variables en comun (el cual explicaremos) y cada modelo tendra la diferencia de la variable indicadora y la variable categorica de si fue o no efectiva la politica. La variable a predecir es "Si hizo o no Cuartena nacional obligatoria. Cero (0) "si hizo"; uno (1) "no hizo".

Variables en comun: "stringency_index"= dice que tan estricta fue la politica. "population_per_100Thousand"= cantidad de poblacion para marcar que los paises tiene diferentes cantidades de pobladores. "population_density_per_100Thousand"= La densidad poblacional el cual no es lo mismo que poblacion.

Para modelar utilizaremos el modelo de Regresion logistica de la libreria Scikit Learn.


### Tasa de reproducción

In [ ]:
XRr=DS_paises_Analisis[["stringency_index","population_density_per_100Thousand","population_per_100Thousand","reproduction_rate","Aument_BajaRR"]]
yRr=DS_paises_Analisis["si_no_Cuarentena"]

In [ ]:
XRr_train, XRr_test, yRr_train, yRr_test = train_test_split(XRr, yRr, test_size = 0.25, random_state=0)

In [ ]:
clf_Rr = LogisticRegression(random_state=0)

In [ ]:
clf_Rr.fit(XRr_train, yRr_train)

In [ ]:
clf_Rr.score(XRr_train,yRr_train)

In [ ]:
clf_Rr.score(XRr_test,yRr_test)

### Capacidad de utilización de las UCI

In [ ]:
X=DS_paises_Analisis[["stringency_index","population_density_per_100Thousand","population_per_100Thousand","icu_patients","Aument_BajaNC"]]
y=DS_paises_Analisis["si_no_Cuarentena"]

In [ ]:
DS_paises_Analisis[["stringency_index","population_density_per_100Thousand","population_per_100Thousand","icu_capacity","Aument_BajaNC"]]

In [ ]:
XSi_train, XSi_test, ySi_train, ySi_test = train_test_split(X, y, test_size = 0.25, random_state=0)

In [ ]:
clf_Si = LogisticRegression(random_state=0)

In [ ]:
np.isnan(X).any()#, np.infinity(X).any()


In [ ]:
clf_Si.fit(XSi_train, ySi_train)

In [ ]:
clf_Si.score(XSi_train,ySi_train)

In [ ]:
clf_Rr.score(XSi_test,ySi_test)

### Niveles de positividad en los testeos

In [ ]:
Xtr=DS_paises_Analisis[["stringency_index","population_density_per_100Thousand","population_per_100Thousand","positive test rate","Aument_Bajatr"]]
ytr=DS_paises_Analisis["si_no_Cuarentena"]

In [ ]:
Xtr_train, Xtr_test, ytr_train, ytr_test = train_test_split(Xtr, ytr, test_size = 0.25, random_state=0)

In [ ]:
clf_tr = LogisticRegression(random_state=0)

In [ ]:
np.isnan(Xtr).any()

In [ ]:
clf_tr.fit(Xtr_train, ytr_train)

In [ ]:
clf_tr.score(Xtr_train,ytr_train)

In [ ]:
clf_Rr.score(Xtr_test,ytr_test)

El mejor modelo es la regresion logistica que considera la "Tasa de reproducción" con accuracy train:0.608 y con accuracy test:0.606.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parametersclf={"penalty":["l1","l2","none","elasticnet"],
               "C":[1.5,1.2,1,0.8,0.5,0.2],
               "solver":["newton-cg","lbfgs","liblinear","sag","saga"]}

In [ ]:
Modelclf_OptHip= LogisticRegression(random_state=0)

In [ ]:
clf_GridSearchCV=GridSearchCV(Modelclf_OptHip,parametersclf,scoring="accuracy",n_jobs=-1,cv=5)

In [ ]:
Fitclf_GridSearchCV=clf_GridSearchCV.fit(XRr_train,yRr_train)
best_modelclf=clf_GridSearchCV.best_estimator_

In [ ]:
Fitclf_GridSearchCV.best_params_

In [ ]:
Model_final_clf=best_modelclf

In [ ]:
Model_final_clf.fit(XRr_train, yRr_train)

In [ ]:
Model_final_clf.score(XRr_train,yRr_train)

In [ ]:
yRr_train_predic=Model_final_clf.predict(XRr_train)
yRr_train_predic

In [ ]:
predic_probas_train=Model_final_clf.predict_proba(XRr_train)

In [ ]:
Model_final_clf.score(XRr_test,yRr_test)

In [ ]:
ynd_test_predic=Model_final_clf.predict(XRr_test)

In [ ]:
predic_probas_test=Model_final_clf.predict_proba(XRr_test)

In [ ]:
(Model_final_clf.score(XRr_test,yRr_test)/Model_final_clf.score(XRr_train,yRr_train))-1

El modelo optimizado mejoro de un accuracy train de:0.608 a 0.764 y el accuracy test de:0.606 a 0.7606. Un incremento no tan notorio ya que el nivel inicial ya se encontraba cercano a 1. a su vez notamos que no tiene casi sesgo ni varianza ya que la diferencia entre ambos es del -0.05%.



In [ ]:
matrix=plot_confusion_matrix(Model_final_clf,XRr_train,yRr_train, cmap=plt.cm.Reds)
matrix.ax_.set_title("Confusion Matrix Dataset Train")
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.gcf().set_size_inches(10,6)
plt.show()

In [ ]:
matrix=plot_confusion_matrix(Model_final_clf,XRr_test,yRr_test, cmap=plt.cm.Reds)
matrix.ax_.set_title("Confusion Matrix Dataset Test")
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.gcf().set_size_inches(10,6)
plt.show()

En las distitas matrices de confusion se ve que lo errores en clasificacion son menores ya que la mayoria estan bien clasificados.

Ahora predeciremos la politica de tres paises que el modelo nunca vio y concluiremos si sirve o no o si parcialmente sirve y habria que hacer mayores modificaciones las cuales quedarian como preguntas a implementar en un futuro.

Para determinar si un pais hizo o no cuarentena nacional la mayoria de las etiqueteas predicar tiene que ser 0 (cero). Utilizaremos a la variable "muertes por cada 100 habitantes" como variable de cruce para la estimación entre los cuales tenemos a Brasil, Croacia y Argentina.

## Experiencias internacionales

### Brasil

In [ ]:
DataS_BR=DSet_paises_Analisis[DSet_paises_Analisis.location=="Brazil"]

In [ ]:
DataS_BR["Aument_BajaRR"] = DataS_BR["new_deaths_per_100Thousand"].map(rango_Aumen_BajaRR)

In [ ]:
print(DataS_BR.shape)
DataS_BR.head(2)

In [ ]:
for col in DataS_BR.columns:
    nulos=DataS_BR[col].isna().sum()
    print(col, "{:.2f}%".format(nulos*100/270))

In [ ]:
X=DataS_BR[["stringency_index","population_density_per_100Thousand","population_per_100Thousand","reproduction_rate","Aument_BajaRR"]]

In [ ]:
y_DS_BR_predic=Model_final_clf.predict(X)
y_DS_BR_predic

In [ ]:
sns.countplot(x=y_DS_BR_predic,data=DataS_BR,palette="plasma")
plt.show()

Utilizamos el modelo y vemos que predice que Brasil no hizo cuarentena en un 100%. Lo cual es correcto.

### Argentina

In [ ]:
DataS_AR=DSet_paises_Analisis[DSet_paises_Analisis.location=="Argentina"]
DataS_AR = DataS_AR.fillna(0)

In [ ]:
DataS_AR["Aument_BajaRR"] = DataS_AR["new_deaths_per_100Thousand"].map(rango_Aumen_BajaRR)

In [ ]:
print(DataS_AR.shape)
DataS_AR.head(2)

In [ ]:
for col in DataS_AR.columns:
    nulos=DataS_AR[col].isna().sum()
    print(col, "{:.2f}%".format(nulos*100/269))

Vemos que la cuarentena ha sido efectiva al ver que la opcion que la opcion cero que, como vimosa previamente, determinaba que la tasa de reproduccion era lo suficientemente alta como para suponer que el freno del COVID-19 fue efectivo.

In [ ]:
sns.countplot(x="Aument_BajaRR",data=DataS_AR)
plt.show()

In [ ]:
X=DataS_AR[["stringency_index","population_density_per_100Thousand","population_per_100Thousand","reproduction_rate","Aument_BajaRR"]]

In [ ]:
y_DS_AR_predic=Model_final_clf.predict(X)
y_DS_AR_predic

In [ ]:
sns.countplot(x=y_DS_AR_predic,data=DataS_AR,palette="plasma")
plt.show()  

Utilizamos el modelo y vemos que predice que Argentina hizo cuarentena nacional ya que tiene una probabilidad de haber hecho cuarentena nacional del 81% vs un 19% de no haberla hecho. Lo cual indica que la estimación realizada se condice con la realidad.

### Croacia

In [ ]:
DataS_CR=DSet_paises_Analisis[DSet_paises_Analisis.location=="Croatia"]

In [ ]:
DataS_CR["Aument_BajaRR"] = DataS_CR["new_deaths_per_100Thousand"].map(rango_Aumen_BajaRR)
DataS_CR = DataS_CR.fillna(0)

In [ ]:
print(DataS_CR.shape)
DataS_CR.head(2)

In [ ]:
for col in DataS_CR.columns:
    nulos=DataS_CR[col].isna().sum()
    print(col, "{:.2f}%".format(nulos*100/271))

In [ ]:
sns.countplot(x="Aument_BajaRR",data=DataS_CR)
plt.show()

In [ ]:
X=DataS_CR[["stringency_index","population_density_per_100Thousand","population_per_100Thousand","reproduction_rate","Aument_BajaRR"]]

In [ ]:
y_DS_CR_predic=Model_final_clf.predict(X)
y_DS_CR_predic

In [ ]:
sns.countplot(x=y_DS_CR_predic,data=DataS_CR,palette="plasma")
plt.show()

En el caso de Croacia, utilizamos nuevamente el modelo y vemos que predice que este país hizo cuarentena nacional ya que tiene una probabilidad de haber hecho cuarentena nacional del 62% vs un 38% de no haberla hecho. Lo cual indica que la estimación realizada se condice a medias con la realidad, ya que la cuarentena realizada fue mucho más corta en comparación con la Argentina.

## Conclusiones 


Concluimos que el mejor modelo desarrollado fue el regresion logistica que se basaba en la variable predictora "Reproduction Rate" esta fue la que mayor accuracy arrojo y luego de optimizar sus hiperparametros llego a un 76% de accuracy.

Tambien concluimos que el indicador mas relevante y certero es el "Reproduction Rate" ya que es el unico que nos dice con certeza si la expansion del virus se esta frenando o no porque los nuevos casos tiene muchos baches en su determinacion como por ejemplo la falta de testeos o los pocos testeos que hace cada país y en relacion a las nuevas muertes es posible errores de carga de datos y se podrian cargar muerte por covid y en realidad son muertes por otras causas a demas de que no todos los contagiados son hospitalizado y no todos los hopitalizados mueren.

En relacion a los modelos, concluimos que es posible hacer un clasificador a partir de una regresion logistica pero aunque como pudimos ver en las pruebas con los tres países nunca visto por el modelo, no logra clasificar un 100% correctamente, creemos que al agregar mas paises de distintas regiones del mundo, respetando la politica seleccionada y agregando mas variables predictoras se podria hacer un clasificador que nos diga que politica implementó cada pais con un mayor grado de precisión y consistencia.



## Preguntas a contestar y corrientes de mejora del trabajo
Cómo reaccionaría el modelo ante cambios en las nuevas cepas de COVID 19 si quisieramos proceder a realizar una separación entre ellas?

Las nuevas variables Omicron y Delta han generado cambios en las formas de testeos ya que han aparecido sintomas que antes no y generando falsos negativos en los testeos, lo cual podría generar nuevos desafíos en la medición del avance de la Pandemia y, por lo tanto, generando incosistencias en los clasificadores 

Que variables se podrían incorporar al modelo y que contemplen los factores socio culturales en él?

Durante el transcurso de la pandemia, los factores psicologicos y de ingreso disponible en los países analizados impactan radicalmente en la decisión de las personas de quedarse en su casa durante la cuarentena como ha pasado en los países latinoamericanos con altos niveles de pobreza. Por otro lado, el enfoque de las cuarentenas obligatorias ha tenido distintos efectos desde la psicologia entre lo que se encuentran los emocionales, por el miedo a la cuarentena y de depresión, que ha generado incluso suicidios y que nos obliga a pensar "que tan beneficiosa es en realidad la cuarentena?" y poder introducirlo en nuestra regresión sería un valioso aporte para las políticas públicas.

Otra posible coriente de análisis sería mediante la utilización de datos georeferenciados sería poder ver las posible ruta de contagios que tenga la enfermedad y poder proyectar los próximos destinos que tendría la Pandemia.

Por último, considero que el cruzamiento con datos de climáticos brindaría mayor robustez para el uso en políticas públicas ya que la gripe es un virus que se tiene una mayor velocidad de propagación durante el invierno. Con lo cual, utilizando todos estos datos se podría realizar un mejor análisis para combatir la Pandemia y utilizar pandemias menos restrictivas y que impliquen un menor daño psicologico en la población.



## Bibliografía Consultado




* Blog el ABC. Estadísticas COVID-19, cuidado con los sesgos! 

* Dirección de asuntos internacionales de Argentina (2021). Medidas restrictivas en el mundo en el marco de la Pandemia por Covid-19.

* Gestión y Economía de la Salud. Cuarentena inteligente: Alternativas de gestión al aislamiento total.

* Lasa et al (2020). Las consecuencias psicólogicas de la COVID-19 y el confienamiento.

* Stoto et al (2022). COVID-19 data are messy: analytic methods for rigorous impact analyses with imperfect
data.

---

